# 1. Function Calling详细流程剖析

In [2]:
import openai
import os
import numpy as np
import pandas as pd
import json
import io
from openai import OpenAI
import inspect

# 从环境变量中获取OpenAI API密钥
openai.api_key = os.getenv("OPENAI_API_KEY")

# 使用API密钥创建OpenAI客户端实例
client = OpenAI(api_key=openai.api_key)

In [3]:
# 定义了一个名为sunwukong_function的函数，该函数接受一个字符串类型的数据集作为参数，并将其转换为pandas DataFrame对象。然后，它将DataFrame中的每个元素乘以10，并将结果转换为JSON格式的字符串返回。
def sunwukong_function(data):
    """
    孙悟空算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：sunwukong_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new * 10
    return json.dumps(res.to_string())

In [4]:
#创建了一个名为df的DataFrame对象，其中包含两列数据x1和x2。
df = pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]})
df

,x1,x2
0,1,3
1,2,4


In [5]:
#将df对象转换为字符串类型。
df_str = df.to_string()
df_str

'   x1  x2\n0   1   3\n1   2   4'

In [6]:
#将df字符串传递给sunwukong_function函数进行计算。最后，它将计算结果打印出来。
result_json=sunwukong_function(df_str)
result_json

'"   x1  x2\\n0  10  30\\n1  20  40"'

In [9]:
# 定义一个工具函数
sunwukong={
        "type": "function",
        "function": {"name": "sunwukong_function",
                      "description": "用于执行孙悟空算法函数，定义了一种特殊的数据集计算过程",
                      "parameters": {"type": "object",
                                     "properties": {"data": {"type": "string",
                                                             "description": "执行孙悟空算法的数据集"},
                                                   },
                                     "required": ["data"],
                                    },
                     }
    }

In [10]:
# 放入工具列表
tools = [sunwukong]

In [11]:
# 这段代码定义了一个名为messages的列表，其中包含两个字典对象。第一个字典对象表示系统角色，其内容为数据集data的描述信息；第二个字典对象表示用户角色，其内容为执行孙悟空算法的请求。
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行孙悟空算法"}
]
# 使用OpenAI API中的chat.completions.create方法来生成响应。该方法接受两个参数：model和messages。model参数指定要使用的模型版本，这里使用的是gpt-3.5-turbo模型；messages参数是要发送给API的消息列表，这里传入的是前面定义的messages列表。

response =  client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)
# 从响应中提取出第一个选择的消息，并将其打印出来。这个消息应该是由API根据输入的数据集和请求生成的孙悟空算法的结果。
response.choices[0].message

ChatCompletionMessage(content='抱歉，我不清楚您指的“孙悟空算法”是什么意思。请问您能提供更多关于该算法的背景或者说明吗？这样我才能帮助您更好地实现您的目标。', role='assistant', function_call=None, tool_calls=None)

In [12]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行孙悟空算法"}
]

In [15]:
# 重新初始化消息列表后，设置tools参数和tool_choice，让大模型自己选择是否使用工具
# 不会直接执行function_call函数。这段代码是调用OpenAI API的chat.completions.create方法，用于生成聊天机器人的回答。其中，model参数指定了使用的模型版本，messages参数传入了要发送给API的消息列表，tools参数传入了工具列表，tool_choice参数指定了选择工具的方式。
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",  
    )
# 查看消息结果，返回内容为空，但是找到返回了工具函数
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z16cI8SX4FkoW71SNV95hjIL', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function')])

In [17]:
# 打印出消息，可以看到消息中已经找到了函数sunwukong_function
first_response = response.choices[0].message
first_response

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z16cI8SX4FkoW71SNV95hjIL', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function')])

In [18]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_z16cI8SX4FkoW71SNV95hjIL', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function')]

In [19]:
#定义了一个名为available_tools的字典，其中包含了可用的工具函数
available_tools =  {
    "sunwukong_function": sunwukong_function,
}

In [20]:
#从API返回的回答中提取出工具调用信息，并遍历每个工具调用。对于每个工具调用，获取函数名、函数参数，并使用这些信息调用相应的工具函数。
tool_calls = response.choices[0].message.tool_calls
 
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_tools[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)
 
# 打印出函数名、函数参数和函数响应。
print(function_name)
print(function_args)
print(function_response)


sunwukong_function
{'data': 'x1  x2\n0   1   3\n1   2   4'}
"   x1  x2\n0  10  30\n1  20  40"


In [23]:
#调用工具函数并获取其响应。它首先使用工具函数名和参数调用相应的工具函数，并将返回值存储在名为function_response的变量中。然后打印出这个响应
function_response = function_to_call(**function_args)
function_response

'"    x2\\nx1    \\n0   10\\n1   20"'

In [21]:
# 追加第一次模型返回结果消息
messages.append(first_response)  
messages

[{'role': 'system',
  'content': '数据集data：   x1  x2\n0   1   3\n1   2   4，数据集以字符串形式呈现'},
 {'role': 'user', 'content': '请在数据集data上执行孙悟空算法'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z16cI8SX4FkoW71SNV95hjIL', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function')])]

In [22]:
# 追加function返回消息
messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
)

In [23]:
messages

[{'role': 'system',
  'content': '数据集data：   x1  x2\n0   1   3\n1   2   4，数据集以字符串形式呈现'},
 {'role': 'user', 'content': '请在数据集data上执行孙悟空算法'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z16cI8SX4FkoW71SNV95hjIL', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function')]),
 {'tool_call_id': 'call_z16cI8SX4FkoW71SNV95hjIL',
  'role': 'tool',
  'name': 'sunwukong_function',
  'content': '"   x1  x2\\n0  10  30\\n1  20  40"'}]

In [24]:
# 再次调用，由大模型重新整理工具函数返回的结果信息（主要是将结果信息整理的更加方便可视化）
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)
 

print(second_response.choices[0].message.content)

孙悟空算法已成功运行在数据集上。数据集经过处理后变为：

```
   x1  x2
0  10  30
1  20  40
```


In [25]:
from openai import OpenAI
import json

openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai.api_key)

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def sunwukong_function(data):
    """
    孙悟空算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：sunwukong_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new['x1'] * 10
    return json.dumps(res.to_string())


df_str=pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]}).to_string

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行孙悟空算法"}  
        ]
    tools = [
    {
        "type": "function",
        "function": {"name": "sunwukong_function",
                      "description": "用于执行孙悟空算法函数，定义了一种特殊的数据集计算过程",
                      "parameters": {"type": "object",
                                     "properties": {"data": {"type": "string",
                                                             "description": "执行孙悟空算法的数据集"},
                                                   },
                                     "required": ["data"],
                                    },
                     }
    }]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "sunwukong_function": sunwukong_function,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
result=run_conversation()

In [26]:
result.choices[0].message.content

'孙悟空算法在数据集上的执行结果为：\n\n   x1  x2\n0   10\n1   20'

In [27]:
# 使用StringIO将字符串转换为文件对象
df_str='\\n    x1\\n0  10\\n1  20\\n.'
data = io.StringIO(df_str)

# 使用read_csv()函数读取数据，并设置第一列为索引
df_new = pd.read_csv(data, sep='\s+', index_col=0)
df_new

,x1\n0,10\n1,20\n.
\n,,,


# 2. Function Calling函数封装

In [55]:
import openai
import os
import numpy as np
import pandas as pd
import json
import io
from openai import OpenAI

openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai.api_key)

In [56]:
response = client.chat.completions.create(
  #model="gpt-4-0613",
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "什么是JSON Schema？"}
  ]
)

response.choices[0].message.content

'JSON Schema是一种用于描述和验证JSON数据结构的规范。它定义了数据的类型、格式、约束和关系，使得可以对JSON数据进行验证和验证。通过JSON Schema，开发人员可以确保数据的完整性、准确性和一致性，以及在不同应用程序和平台之间的数据交换的有效性。JSON Schema可以被用来验证输入数据、生成文档和测试数据等各种用途。'

In [57]:
def sunwukong_function(data):
    """
    孙悟空算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：sunwukong_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new * 10
    return json.dumps(res.to_string())

In [58]:
# 创建一个DataFrame
df = pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]})

df_str = df.to_string()

data = io.StringIO(df_str)

df_new = pd.read_csv(data, sep='\s+', index_col=0)

In [59]:
sunwukong={
        "type": "function",
        "function": {"name": "sunwukong_function",
                      "description": "用于执行孙悟空算法函数，定义了一种特殊的数据集计算过程",
                      "parameters": {"type": "object",
                                     "properties": {"data": {"type": "string",
                                                             "description": "执行孙悟空算法的数据集"},
                                                   },
                                     "required": ["data"],
                                    },
                     }
        }

In [60]:
tools = [sunwukong]

In [61]:
available_tools =  {
    "sunwukong_function": sunwukong_function,
}

In [62]:
import inspect
# 取出注释信息
print(inspect.getdoc(sunwukong_function))

孙悟空算法函数，该函数定义了数据集计算过程
:param data: 必要参数，表示带入计算的数据表，用字符串进行表示
:return：sunwukong_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象


In [63]:
function_description = inspect.getdoc(sunwukong_function)

In [64]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "以下是孙悟空函数的函数说明：%s" % function_description},
    {"role": "user", "content": "请帮我编写一个JSON Schema对象，用于说明孙悟空函数的参数输入规范。输出结果要求是JSON Schema格式的JONS类型对象，不需要任何前后修饰语句。"}
  ]
)
# 使用gpt3.5发现有时候生成正确，但是有时候生成的json信息还是有些缺少，gpt.4会更稳定
response.choices[0].message.content

'{\n  "type": "object",\n  "required": ["data"],\n  "properties": {\n    "data": {\n      "type": "string",\n      "description": "Represents the data table to be calculated"\n    }\n  }\n}'

In [65]:
# 将变量 response.choices[0].message.content 中的字符串中的 "" 和 "json" 替换为空字符串
r=response.choices[0].message.content.replace("```","").replace("json","")

In [42]:
#s='{\n  "$schema": "http://json-schema.org/draft-07/schema#",\n  "type": "object",\n  "properties": {\n    "data": {\n      "type": "string"\n    }\n  },\n  "required": ["data"]\n}'

In [66]:
json.loads(r)

{'type': 'object',
 'required': ['data'],
 'properties': {'data': {'type': 'string',
   'description': 'Represents the data table to be calculated'}}}

In [67]:
# 打印悟空函数的json格式，与上面模型生成的json对比
sunwukong

{'type': 'function',
 'function': {'name': 'sunwukong_function',
  'description': '用于执行孙悟空算法函数，定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string', 'description': '执行孙悟空算法的数据集'}},
   'required': ['data']}}}

In [68]:
sunwukong['function']['parameters']

{'type': 'object',
 'properties': {'data': {'type': 'string', 'description': '执行孙悟空算法的数据集'}},
 'required': ['data']}

In [69]:
system_prompt = '以下是某的函数说明：%s' % function_description
user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
               1.字典总共有三个键值对；\
               2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
               3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
               4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
               5.输出结果必须是一个JSON格式的字典，且不需要任何前后修饰语句' % function_name

In [70]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]
)
response.choices[0].message.content

'{\n    "name": "sunwukong_function",\n    "description": "孙悟空算法函数，该函数定义了数据集计算过程",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "data": {\n                "type": "string",\n                "description": "必要参数，表示带入计算的数据表，用字符串进行表示"\n            }\n        },\n        "required": ["data"]\n    }\n}'

In [71]:
json_function_description=json.loads(response.choices[0].message.content.replace("```","").replace("json",""))

In [72]:
json_function_description

{'name': 'sunwukong_function',
 'description': '孙悟空算法函数，该函数定义了数据集计算过程',
 'parameters': {'type': 'object',
  'properties': {'data': {'type': 'string',
    'description': '必要参数，表示带入计算的数据表，用字符串进行表示'}},
  'required': ['data']}}

In [74]:
# 输出悟空函数，和生成的函数信息对比
sunwukong

{'type': 'function',
 'function': {'name': 'sunwukong_function',
  'description': '用于执行孙悟空算法函数，定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string', 'description': '执行孙悟空算法的数据集'}},
   'required': ['data']}}}

In [75]:
# 补充缺少的部分信息
json_str={"type": "function","function":json_function_description}
json_str

{'type': 'function',
 'function': {'name': 'sunwukong_function',
  'description': '孙悟空算法函数，该函数定义了数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string',
     'description': '必要参数，表示带入计算的数据表，用字符串进行表示'}},
   'required': ['data']}}}

In [76]:
# 再次输出悟空函数，进行对比，基本上已经一摸一样了
sunwukong

{'type': 'function',
 'function': {'name': 'sunwukong_function',
  'description': '用于执行孙悟空算法函数，定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string', 'description': '执行孙悟空算法的数据集'}},
   'required': ['data']}}}

### 定义自动输出function 参数的函数

In [81]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            system_prompt = '以下是某的函数说明：%s' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
               1.字典总共有三个键值对；\
               2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
               3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
               4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
               5.输出结果必须是一个JSON格式的字典，且不需要任何前后修饰语句' % function_name

            response = client.chat.completions.create(
                              model="gpt-3.5-turbo",
                              messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_function_description=json.loads(response.choices[0].message.content.replace("```","").replace("json",""))
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [82]:
functions_list = [sunwukong_function]
tools = auto_functions(functions_list)

In [83]:
tools

[{'type': 'function',
  'function': {'name': 'sunwukong_function',
   'description': '孙悟空算法函数，该函数定义了数据集计算过程',
   'parameters': {'type': 'object',
    'properties': {'data': {'type': 'string',
      'description': '表示带入计算的数据表，用字符串进行表示'}},
    'required': ['data']}}}]

In [84]:
df_str = pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]}).to_string()
df_str

'   x1  x2\n0   1   3\n1   2   4'

In [85]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行孙悟空算法"}
]

In [86]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",  
    )
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0YPmqwOXDhBYa4AvDzye2tc8', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function')])

In [87]:
#在定义一个工具函数，一起测试
def tangseng_function(data):
    """
    唐僧算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：tangseng_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new * 1000000
    return json.dumps(res.to_string())

In [88]:
functions_list=[sunwukong_function,tangseng_function]

In [93]:
# 使用gpt3.5发现有时候生成正确，但是有时候生成的json信息还是有些缺少，gpt.4会更稳定
tools = auto_functions(functions_list)
tools

[{'type': 'function',
  'function': {'name': 'sunwukong_function',
   'description': '孙悟空算法函数，该函数定义了数据集计算过程',
   'parameters': {'type': 'object',
    'properties': {'data': {'type': 'string', 'description': '表示带入计算的数据表'}},
    'required': ['data']}}},
 {'type': 'function',
  'function': {'name': 'tangseng_function',
   'description': '唐僧算法函数，该函数定义了数据集计算过程',
   'parameters': {'type': 'object',
    'properties': {'data': {'type': 'string',
      'description': '必要参数，表示带入计算的数据表，用字符串进行表示'}},
    'required': ['data']}}}]

In [94]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行唐僧算法函数"}
]
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto", 
    )
response.choices[0].message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ovj6ay0nmjLlXoufWEcwBDy9', function=Function(arguments='{"data":"   x1  x2\\n0   1   3\\n1   2   4"}', name='tangseng_function'), type='function')])

# 3. 封装调用2轮response的函数

In [95]:
functions_list

[<function __main__.sunwukong_function(data)>,
 <function __main__.tangseng_function(data)>]

In [96]:
def run_conversation(messages, functions_list=None, model="gpt-3.5-turbo"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为gpt-3.5-turbo
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message


        tool_calls = response_message.tool_calls

        if tool_calls:

            messages.append(response_message) 
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
            ) 
            # 获取最终结果
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [97]:
df_str = pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]}).to_string()
df_str

'   x1  x2\n0   1   3\n1   2   4'

In [98]:
## 测试一下函数（1）
messages = [
        {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
        {"role": "user", "content": '请在data上执行唐僧算法函数'}]


In [99]:
run_conversation(messages = messages, functions_list = functions_list)

'唐僧算法已成功应用于数据集，并将数据集中的值乘以1000000。现在x1和x2的值已被调整为新的值。'

In [100]:
## 测试一下函数（2）

messages = [
        {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
        {"role": "user", "content": '请在data上执行孙悟空算法函数'}]

In [102]:
run_conversation(messages = messages, functions_list = functions_list)

'孙悟空算法已成功应用于该数据集，处理后的数据集如下所示：  \n\n|    |   x1 |   x2 |\n|---:|-----:|-----:|\n|  0 |   10 |   30 |\n|  1 |   20 |   40 |'

'已在数据集上成功执行了孙悟空算法函数，修改后的数据集为：\n\n```\n   x1  x2\n0  10  30\n1  20  40\n```'

In [82]:
## 测试一下函数（3）

messages = [
        {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
        {"role": "user", "content": '请解释一下data数据集'}]

In [83]:
run_conversation(messages = messages, functions_list = functions_list)

'数据集data是一个包含两列x1和x2的数据集，每一行表示一个数据点。例如，第一行的数据是x1=1，x2=3，第二行的数据是x1=2，x2=4。数据集以字符串形式呈现，可以使用这些数据点进行进一步的计算和分析。'

# 4. 增加多轮对话的效果

In [107]:
import openai
import os
import numpy as np
import pandas as pd
import json
import io
from openai import OpenAI
import inspect
# 定义api key
openai.api_key = os.getenv("OPENAI_API_KEY")

#创建OpenAI客户端
client = OpenAI(api_key=openai.api_key)

In [108]:
def sunwukong_function(data):
    """
    孙悟空算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：sunwukong_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new * 10
    return json.dumps(res.to_string())

In [109]:
def tangseng_function(data):
    """
    唐僧算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：tangseng_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new * 1000000
    return json.dumps(res.to_string())

In [110]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            system_prompt = '以下是某的函数说明：%s' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="gpt-3.5-turbo",
                              messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_function_description=json.loads(response.choices[0].message.content.replace("```","").replace("json",""))
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [111]:
def run_conversation(messages, functions_list=None, model="gpt-3.5-turbo"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为gpt-3.5-turbo
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message


        tool_calls = response_message.tool_calls

        if tool_calls:

            messages.append(response_message) 
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
            ) 
            # 获取最终结果
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [112]:
def chat_with_model(functions_list=None, 
                    prompt="你好", 
                    model="gpt-3.5-turbo", 
                    system_message=[{"role": "system", "content": "你是小智助手。"}]):
    
    messages = system_message
    messages.append({"role": "user", "content": prompt})
    
    while True:           
        answer = run_conversation(messages=messages, 
                                    functions_list=functions_list, 
                                    model=model)
        
        print(f"智能助手回答: {answer}")
        
        
        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            break

        # 记录用户回答
        messages.append({"role": "user", "content": user_input})

In [113]:
functions_list=[sunwukong_function,tangseng_function]

In [116]:
chat_with_model(functions_list,prompt="你好")

智能助手回答: 你好，请问有什么可以帮助你的吗？
智能助手回答: 你好！有什么可以帮助你的吗？
智能助手回答: 1 + 2 等于 3。
智能助手回答: 孙悟空算法函数是一个用来执行数据集计算过程的函数。通过提供数据集作为参数，该函数可以执行特定的计算任务。
智能助手回答: 执行了孙悟空算法后，数据集data变为：'   x1  x2\n0  10  30\n1  20  40'


In [1]:
user_input = input("您还有其他问题吗？(输入退出以结束对话): ")